In [1]:
%load_ext autoreload
%autoreload 2
from dimcat import (
    Corpus,  
    Pipeline,
    IsAnnotatedFilter,
    CorpusGrouper, 
    PieceGrouper, 
    ModeGrouper, 
    ChordSymbolBigrams, 
    ChordSymbolUnigrams,
    LocalKeySlicer,
)
import pandas as pd

In [2]:
corpus_path = "~/romantic_piano_corpus"
corpus = Corpus(directory=corpus_path)
corpus.data

1154 files.
KEY                       -> EXTENSIONS
---------------------------------------
beethoven_piano_sonatas   -> {'.mscx': 87, '.tsv': 239}
chopin_mazurkas           -> {'.mscx': 55, '.tsv': 166}
debussy_suite_bergamasque -> {'.mscx': 4, '.tsv': 13}
dvorak_silhouettes        -> {'.mscx': 12, '.tsv': 37}
grieg_lyrical_pieces      -> {'.mscx': 66, '.tsv': 199}
liszt_pelerinage          -> {'.mscx': 19, '.tsv': 77}
medtner_tales             -> {'.mscx': 19, '.tsv': 59}
schumann_kinderszenen     -> {'.mscx': 13, '.tsv': 40}
tchaikovsky_seasons       -> {'.mscx': 12, '.tsv': 37}

None of the 287 score files have been parsed.

All 867 tabular files have been parsed, 283 of them as Annotations object(s).
KEY                       -> ANNOTATION LAYERS
----------------------------------------------
beethoven_piano_sonatas   -> staff  voice  label_type  color  
                          -> 2      1      0 (dcml)    default    20720
                          ->        2      0 (dcml)    d

In [3]:
annotated = IsAnnotatedFilter().process_data(corpus)
print(f"Before: {len(corpus.indices[()])} IDs, after filtering: {len(annotated.indices[()])}")

Before: 287 IDs, after filtering: 264


# Tokens

All symbols, independent of the local key (the mode of which changes their semantics).

In [4]:
for group, dict_of_dfs in annotated.iter_facet("expanded", concatenate=True):
    for tuple_of_all_id_tuples, all_annotations in dict_of_dfs.items():
        pass
all_annotations.sample(20)

quarterbeats  \
                                                                  interval                        
medtner_tales             op42n03                                 [105.5, 106.5)          211/2   
beethoven_piano_sonatas   05-3                                    [105.5, 106.5)          211/2   
                          07-1                                    [1017.0, 1020.0)         1017   
                          02-1                                    [335.5, 336.5)          671/2   
                          03-2                                    [129.0, 130.0)            129   
                          17-1                                    [789.5, 790.5)         1579/2   
                          26-3                                    [535.5, 536.5)         1071/2   
medtner_tales             op42n01                                 [102.5, 103.0)          205/2   
tchaikovsky_seasons       op37a06                                 [101.0, 102.0)            101   
beethoven_piano_sonatas   06-3                                    [76.5, 78.5)            153/2   
debussy_suite_bergamasque l075-04_suite_passepied                 [494.0, 496.0)            494   
liszt_pelerinage          160.09_Les_Cloches_de_Geneve_(Nocturne) [111.0, 112.5)            111   
medtner_tales             op34n02                                 [19.0, 19.5)               19   
chopin_mazurkas           BI115-4op33-4                           [639.0, 640.0)            639   
beethoven_piano_sonatas   23-1                                    [169.0, 170.5)            169   
grieg_lyrical_pieces      op68n04                                 [39.0, 42.0)               39   
medtner_tales             op48n01                                 [245.0, 246.0)            245   
tchaikovsky_seasons       op37a03                                 [10.0, 11.0)               10   
beethoven_piano_sonatas   02-4                                    [535.0, 536.0)            535   
grieg_lyrical_pieces      op68n05                                 [78.0, 79.0)               78   

                                                                                   duration_qb  \
                                                                  interval                       
medtner_tales             op42n03                                 [105.5, 106.5)           1.0   
beethoven_piano_sonatas   05-3                                    [105.5, 106.5)           1.0   
                          07-1                                    [1017.0, 1020.0)         3.0   
                          02-1                                    [335.5, 336.5)           1.0   
                          03-2                                    [129.0, 130.0)           1.0   
                          17-1                                    [789.5, 790.5)           1.0   
                          26-3                                    [535.5, 536.5)           1.0   
medtner_tales             op42n01                                 [102.5, 103.0)           0.5   
tchaikovsky_seasons       op37a06                                 [101.0, 102.0)           1.0   
beethoven_piano_sonatas   06-3                                    [76.5, 78.5)             2.0   
debussy_suite_bergamasque l075-04_suite_passepied                 [494.0, 496.0)           2.0   
liszt_pelerinage          160.09_Les_Cloches_de_Geneve_(Nocturne) [111.0, 112.5)           1.5   
medtner_tales             op34n02                                 [19.0, 19.5)             0.5   
chopin_mazurkas           BI115-4op33-4                           [639.0, 640.0)           1.0   
beethoven_piano_sonatas   23-1                                    [169.0, 170.5)           1.5   
grieg_lyrical_pieces      op68n04                                 [39.0, 42.0)             3.0   
medtner_tales             op48n01                                 [245.0, 246.0)           1.0   
tchaikovsky_seasons       op37a03         

In [5]:
no_chord = all_annotations.root.isna()
print(f"Concatenated annotation tables contains {all_annotations.shape[0]} rows. {no_chord.sum()} of them are not chords. Their values are:")
all_annotations.label[no_chord].value_counts(dropna=False).to_dict()

Concatenated annotation tables contains 57566 rows. 677 of them are not chords. Their values are:


{'{': 640, '}': 28, '|PAC}': 6, '|HC': 2, '|PAC': 1}

In [6]:
all_chords = all_annotations[~no_chord]
print(f"Corpus contains {all_chords.shape[0]} tokens and {len(all_chords.chord.unique())} types over {len(tuple_of_all_id_tuples)} documents.")

Corpus contains 56889 tokens and 3115 types over 264 documents.


In [7]:
#from ms3 import write_tsv
#write_tsv(all_annotations[all_annotations.pedalend.notna()], './issues/pedalpoints.tsv', pre_process=False)

# Unigrams
For computing unigram statistics, the tokens need to be grouped by their occurrence within a major or a minor key because this changes their meaning. To that aim, the annotated corpus needs to be sliced into contiguous localkey segments which are then grouped into a major (`is_minor=False`) and a minor group.

In [8]:
localkey_slices = LocalKeySlicer().process_data(annotated)

No localkey known for ('liszt_pelerinage', '162.03_Tarantella_da_Guillaume_Louis_Cottrau._Presto_e_canzone_napolitana'). Skipping.


In [9]:
mode_slices = ModeGrouper().process_data(localkey_slices)

## Whole dataset

In [10]:
unigrams = ChordSymbolUnigrams(once_per_group=True).process_data(mode_slices)

In [11]:
unigrams.get()

localkey_is_minor  chord 
False              I         5181
                   V7        2581
                   V         2158
                   I6        1742
                   IV         972
                             ... 
True               i/#iii       1
                   V2/#VI       1
                   I6/#VI       1
                   i6/#vi       1
                   VI6(9)       1
Name: count, Length: 3681, dtype: int64

In [12]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR UNIGRAMS
1990 types, 35328 tokens
chord
I        5181
V7       2581
V        2158
I6       1742
IV        972
V(64)     716
V43       664
V2        643
V6        629
vi        609
ii        566
ii6       544
V65       541
IV6       466
I64       436
V7(9)     358
viio6     347
V7/V      324
ii7       278
V7/IV     229
MINOR UNIGRAMS
1691 types, 20850 tokens
chord
i          2658
V          1389
V7         1135
i6          889
iv          483
V(64)       441
V6          348
VI          321
iv6         308
V43         297
i64         239
III         231
V65         218
V2          209
#viio43     207
I           202
v           170
bII         161
V7/III      158
viio7/V     152


## Per corpus

In [13]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [14]:
corpus_wise_unigrams.get()

localkey_is_minor  corpus                   chord   
False              beethoven_piano_sonatas  I           2232
                                            V           1189
                                            I6          1125
                                            V7          1041
                                            V6           468
                                                        ... 
True               tchaikovsky_seasons      i(4)/v         1
                                            i/v            1
                                            ii64           1
                                            v6/iv          1
                                            ii%65(2)       1
Name: count, Length: 6637, dtype: int64

In [15]:
for (is_minor, corpus), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR unigrams (842 types, 15275 tokens)
chord
I     2232
V     1189
I6    1125
V7    1041
V6     468
chopin_mazurkas MAJOR unigrams (451 types, 5256 tokens)
chord
I       949
V7      659
V       252
IV      189
V7/V    128
debussy_suite_bergamasque MAJOR unigrams (106 types, 363 tokens)
chord
I         28
V7        22
iii       19
V7(+2)    12
IV6       12
dvorak_silhouettes MAJOR unigrams (140 types, 1067 tokens)
chord
I     254
I6    103
V7     89
V      79
IV     46
grieg_lyrical_pieces MAJOR unigrams (659 types, 4882 tokens)
chord
I     566
V7    264
V     224
IV    127
vi    116
liszt_pelerinage MAJOR unigrams (531 types, 3173 tokens)
chord
I     453
V7    234
V     137
IV     99
I6     94
medtner_tales MAJOR unigrams (636 types, 2864 tokens)
chord
I     263
V     139
V7    122
I6     83
IV     61
schumann_kinderszenen MAJOR unigrams (114 types, 714 tokens)
chord
I      106
I6      69
V       65
V7      50
V43     30
tchaikovsky_seasons MAJOR unigrams (194

## Per piece

In [16]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [17]:
piece_wise_unigrams.get()

localkey_is_minor  fname    chord 
False              01-1     V7        11
                            I6         8
                            I          7
                            ii6        5
                            V          5
                                      ..
True               op71n07  ii%65      3
                            VIM43      2
                            ii%7       2
                            V(964)     2
                            #viio2     1
Name: count, Length: 15918, dtype: int64

# Bigrams

## Whole dataset

In [18]:
bigrams = ChordSymbolBigrams(once_per_group=True).process_data(mode_slices)

In [19]:
bigrams.get()

localkey_is_minor  from        to    
False              V7          I         1238
                   V           I          578
                   I           V          477
                               V7         422
                   V2          I6         344
                                         ... 
True               V7(+4)/V    IM65/V       1
                   V7(+2)/iv   iv(2)        1
                               V7/iv        1
                   V7(+2)/VII  i            1
                   vo7         i            1
Name: count, Length: 15479, dtype: int64

In [20]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in bigrams.iter():
    print(f"{modes[is_minor]} BIGRAMS\n{ugs.shape[0]} transition types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

MAJOR BIGRAMS
8771 transition types, 35720 tokens
from   to 
V7     I      1238
V      I       578
I      V       477
       V7      422
V2     I6      344
V(64)  V7      309
I      I6      293
I6     I       288
V65    I       267
I      nan     251
       IV      230
V      V7      224
V43    I       213
I      V43     204
       I       175
IV     I       165
V6     I       159
I      V6      148
V      I6      146
V(64)  V       142
MINOR BIGRAMS
6708 transition types, 21128 tokens
from   to 
V7     i      558
V      i      394
i      V      240
       V7     191
       i6     178
       nan    160
i6     i      145
V(64)  V7     143
V      V7     136
V(64)  V      134
V43    i      131
V65    i      128
V6     i      116
i      V6     101
V2     i6      94
i      iv      86
       V43     82
       i       74
V      nan     68
i      VI      66


## Per corpus

In [21]:
corpus_wise_bigrams = Pipeline([CorpusGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

In [22]:
corpus_wise_bigrams.get()

localkey_is_minor  corpus                   from    to    
False              beethoven_piano_sonatas  V7      I         532
                                            V       I         318
                                            I       V         248
                                            V2      I6        238
                                            V(64)   V7        213
                                                             ... 
True               tchaikovsky_seasons      Fr6/iv  I(4)        1
                                            i       V7(#9)      1
                                                    V7(#2)      1
                                            Ger6    V65         1
                                            i6      Fr6/iv      1
Name: count, Length: 19884, dtype: int64

In [23]:
for (is_minor, corpus), ugs in corpus_wise_bigrams.iter():
    print(f"{corpus} {modes[is_minor]} bigrams ({ugs.shape[0]} transition types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

beethoven_piano_sonatas MAJOR bigrams (3638 transition types, 15472 tokens)
from   to
V7     I     532
V      I     318
I      V     248
V2     I6    238
V(64)  V7    213
chopin_mazurkas MAJOR bigrams (1453 transition types, 5322 tokens)
from  to
V7    I     337
I     V7    105
      IV     94
      V      75
IV    I      66
debussy_suite_bergamasque MAJOR bigrams (252 transition types, 362 tokens)
from  to   
I     iii      8
V7    I        7
iii6  V7       5
iii   bIII6    5
IV    iii      4
dvorak_silhouettes MAJOR bigrams (363 transition types, 1073 tokens)
from  to
V7    I     63
I     V7    45
V     I     41
V2    I6    29
I     I6    22
grieg_lyrical_pieces MAJOR bigrams (1780 transition types, 4929 tokens)
from  to
V7    I     101
I     I      44
      V      39
V     I      38
I     V7     33
liszt_pelerinage MAJOR bigrams (1444 transition types, 3192 tokens)
from  to
V7    I     85
I     V     43
      V7    41
I(9)  I     41
V     I     40
medtner_tales MAJOR bigrams (1641 t

## Per piece

In [24]:
piece_wise_bigrams = Pipeline([PieceGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

('beethoven_piano_sonatas', '23-2', Interval(190.0, 194.0, closed='left')): DataFrame has only one row, cannot compute bigram.
Group '(True, '23-2')' will be missing from the processed data.


In [25]:
piece_wise_bigrams.get()

localkey_is_minor  fname    from     to    
False              01-1     V(64)    V7        4
                            viio7/V  V(64)     3
                            ii6(2)   ii6       3
                            V65/V    V         3
                            V7(+b9)  V7        3
                                              ..
True               op71n07  V(964)   ii%65     2
                            VIM43    ii%7      2
                            ii%65    VIM43     2
                            ii%7     V(964)    2
                            ii%65    #viio2    1
Name: count, Length: 28391, dtype: int64